In [1]:
import sys
sys.path.append("/kaggle/input/marigold_model/transformers/default/2/marigold_source")
from Marigold.marigold.marigold_depth_pipeline import MarigoldDepthPipeline

2025-08-14 21:15:59.197384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755206159.385204      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755206159.437376      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os, gc, cv2, torch, h5py, shutil
import numpy as np
from PIL import Image
from tqdm import tqdm

# ===== 1) 模型加载 =====
model_path = "/kaggle/input/marigold_model/transformers/default/2/marigold_model"
pipe = MarigoldDepthPipeline.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# ===== 2) 路径与数据加载 =====
mat_path = '/kaggle/input/nyuv2-dataset/nyu_depth_v2_labeled.mat'
test_idx_path = '/kaggle/input/nyu2-test/nyu2_test.txt'

OUT_ROOT = '/kaggle/working/download/nyuv2_marigold'
out_pred_raw_npy   = os.path.join(OUT_ROOT, 'pred_raw_npy')       # 相对深度 .npy
out_pred_aligned   = os.path.join(OUT_ROOT, 'pred_aligned_npy')   # （可选）scale-only 对齐 .npy
out_gt_npy         = os.path.join(OUT_ROOT, 'gt_npy')             # GT 米制 .npy
out_gt_png16       = os.path.join(OUT_ROOT, 'gt_png16')           # GT 毫米 16-bit PNG
for d in [out_pred_raw_npy, out_pred_aligned, out_gt_npy, out_gt_png16]:
    os.makedirs(d, exist_ok=True)

# 读取 NYUv2 数据
with h5py.File(mat_path, 'r') as f:
    images = f['images'][:]   # 原始: (1449, 3, 640, 480)
    depths = f['depths'][:]   # 原始: (1449, 640, 480)

# 转为 [H, W, 3, N] 与 [H, W, N]（H=480, W=640, N=1449）
images = np.transpose(images, (3, 2, 1, 0)).astype(np.uint8)
depths = np.transpose(depths, (2, 1, 0)).astype(np.float32)

# 官方 test 划分（1-based → 0-based）
idx_list = np.loadtxt(test_idx_path).astype(int) - 1

# ===== 3) 工具 =====
def npy_save(path, arr):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    np.save(path, arr.astype(np.float32))

def mm_png_save(path, depth_m):
    """保存为 16-bit PNG（毫米）"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    depth_mm = np.clip(depth_m * 1000.0, 0, 65535).astype(np.uint16)
    cv2.imwrite(path, depth_mm)

def scale_match(pred, gt, mask):
    """每张图纯缩放对齐（scale-only）"""
    v = mask & np.isfinite(gt) & np.isfinite(pred) & (gt > 0)
    if not np.any(v):
        return 1.0
    p, g = pred[v], gt[v]
    den = float((p**2).sum())
    return float((g * p).sum() / den) if den > 0 else 1.0

# ===== 4) 主循环：只保存 预测 + GT =====
saved = 0
for idx in tqdm(idx_list, desc="Saving NYUv2 (Marigold)"):
    try:
        rgb = images[..., idx]           # (480,640,3) uint8
        gt  = depths[..., idx]           # (480,640)   float32 (米)
        mask = (gt > 0) & np.isfinite(gt)

        # 推理（Marigold 相对深度）
        with torch.no_grad():
            result = pipe(
                Image.fromarray(rgb),
                denoising_steps=4,
                ensemble_size=1,
                processing_res=768,
                match_input_res=True,
                batch_size=0,
                show_progress_bar=False,
            )
        pred_rel = np.asarray(result.depth_np, dtype=np.float32)

        # 尺寸对齐到 GT
        if pred_rel.shape != gt.shape:
            pred_rel = cv2.resize(pred_rel, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

        # （可选）scale-only 对齐（用于快速对比；统一评测时仍用 raw + alignment="ls"）
        s = scale_match(pred_rel, gt, mask)
        pred_aligned = pred_rel * s

        stem = f"{int(idx):04d}"

        # 保存预测
        npy_save(os.path.join(out_pred_raw_npy, f"{stem}.npy"), pred_rel)
        npy_save(os.path.join(out_pred_aligned, f"{stem}.npy"), pred_aligned)  #可注释

        # 保存 GT
        npy_save(os.path.join(out_gt_npy,   f"{stem}.npy"), gt)
        mm_png_save(os.path.join(out_gt_png16, f"{stem}.png"), gt)

        saved += 1

        # 清理
        del rgb, gt, pred_rel, pred_aligned, result, mask
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    except Exception:
        # 静默跳过异常样本
        continue

print(f"[NYUv2 Marigold SAVE] saved pairs: {saved} / {len(idx_list)}")

# ===== 5) 打包下载 =====
zip_base = "/kaggle/working/marigold_nyuv2_preds_and_gt"
shutil.make_archive(zip_base, "zip", OUT_ROOT)
print(f"打包完成: {zip_base}.zip")


The config attributes {'prediction_type': 'depth'} were passed to MarigoldDepthPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'prediction_type': 'depth'} are not expected by MarigoldDepthPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Saving NYUv2 (Marigold): 100%|██████████| 666/666 [22:27<00:00,  2.02s/it]


[NYUv2 Marigold SAVE] saved pairs: 666 / 666
打包完成: /kaggle/working/marigold_nyuv2_preds_and_gt.zip
